In [1]:
import pandas as pd
import requests

In [2]:
app_token = "Rl5BUiRawpr4H2LA9OQeKB47L"

In [3]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [4]:
import geopandas as gpd

# Data Preprocessing & Data Cleaning & Data Filtering

### 311 Complaints

In [5]:
complaints_data = requests.get(url="https://data.cityofnewyork.us/resource/erm2-nwe9.json",
                               params={"$$app_token": app_token, "$where": "created_date between '2018-10-01T00:00:00' and '2018-10-10T23:59:59.9999'", "$limit": 99999999}).json()
complaints_df = pd.DataFrame(complaints_data)

In [6]:
from datetime import datetime, timedelta

def generate_month_ranges(start_date, end_date):
    current = start_date
    print("initial: {}".format(current))
    temp = 0
    while current < end_date:
        temp += 1
        month_end = current.replace(day=1) + timedelta(days=32)
        month_end = month_end.replace(day=1) - timedelta(days=1) # 9 30
        yield (current, month_end)
        current = month_end + timedelta(days=1)
        if temp == 100:
            break

# 设置起始和结束日期
start_date = datetime(2018, 10, 1)
end_date = datetime(2023, 9, 30)

# 生成月份范围
month_ranges = list(generate_month_ranges(start_date, end_date))

initial: 2018-10-01 00:00:00


In [7]:
import os

for start, end in month_ranges:
    year = start.year
    csv_file = f"data/complaints_data_{year}.csv"
    print(f"Downloading data for the period: {start.strftime('%Y-%m-%d')} to {end.strftime('%Y-%m-%d')}")

    query = f"created_date between '{start.strftime('%Y-%m-%dT%H:%M:%S')}' and '{end.strftime('%Y-%m-%dT%H:%M:%S')}'"
    response = requests.get(
        url="https://data.cityofnewyork.us/resource/erm2-nwe9.json",
        params={"$$app_token": app_token, "$where": query, "$limit": 999999,
                "$select": "unique_key, created_date, closed_date, agency,  complaint_type, descriptor, location_type, incident_zip, latitude, longitude, borough"
                }
    )
    data = response.json()
    batch_df = pd.DataFrame(data)

    # 根据年份写入或追加到对应的CSV文件
    mode = 'a' if os.path.exists(csv_file) else 'w'
    batch_df.to_csv(csv_file, mode=mode, index=False, header=not os.path.exists(csv_file))

print("Data download complete.")

Data download complete.


In [8]:
complaints_2021_df = pd.read_csv("data/complaints_data_2021.csv")

/var/folders/98/l9_9vlzx4vs8g303whfvnnjc0000gn/T/ipykernel_17023/402780476.py:1: DtypeWarning: Columns (7,10) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_2021_df = pd.read_csv("data/complaints_data_2021.csv")


In [9]:
complaints_2021_df.shape

(6230090, 11)

In [10]:
complaints_2021_df.columns

Index(['unique_key', 'created_date', 'closed_date', 'agency', 'complaint_type',
       'descriptor', 'location_type', 'incident_zip', 'latitude', 'longitude',
       'borough'],
      dtype='object')

In [11]:
complaints_2021_df.head()

,unique_key,created_date,closed_date,agency,complaint_type,descriptor,location_type,incident_zip,latitude,longitude,borough
0,49669476,2021-01-31T00:00:00.000,2021-02-03T00:00:00.000,DSNY,Request Large Bulky Item Collection,Request Large Bulky Item Collection,Sidewalk,11207.0,40.675269,-73.890711,BROOKLYN
1,49674146,2021-01-30T23:59:34.000,2021-01-31T08:55:13.000,NYPD,Illegal Parking,Commercial Overnight Parking,Street/Sidewalk,10462.0,40.853203,-73.860282,BRONX
2,49671061,2021-01-30T23:59:30.000,2021-01-31T00:41:08.000,NYPD,Noise - Residential,Loud Talking,Residential Building/House,11233.0,40.680672,-73.907492,BROOKLYN
3,49669720,2021-01-30T23:59:18.000,2021-01-31T01:11:00.000,NYPD,Noise - Commercial,Loud Music/Party,Store/Commercial,10034.0,40.864086,-73.918004,MANHATTAN
4,49669589,2021-01-30T23:59:04.000,2021-01-31T00:37:46.000,NYPD,Non-Emergency Police Matter,Trespassing,Residential Building/House,11212.0,40.671226,-73.909185,BROOKLYN


### 2015 Tree census

In [12]:
tree_data = requests.get(url="https://data.cityofnewyork.us/resource/5rq2-4hqu.json",
                         params={"$$app_token": app_token, "$limit": 99999999999999999999}).json()
tree_df = pd.DataFrame(tree_data)

In [13]:
tree_df.shape

(683788, 42)

In [14]:
tree_df.head()

,created_at,tree_id,block_id,the_geom,tree_dbh,stump_diam,curb_loc,status,health,spc_latin,...,st_assem,st_senate,nta,nta_name,boro_ct,state,latitude,longitude,x_sp,y_sp
0,08/27/2015,180683,348711,"{'type': 'Point', 'coordinates': [-73.84421521...",3,0,OnCurb,Alive,Fair,Acer rubrum,...,28,16,QN17,Forest Hills,4073900,New York,40.72309177,-73.84421522,1027431.14821,202756.768749
1,09/03/2015,200540,315986,"{'type': 'Point', 'coordinates': [-73.81867945...",21,0,OnCurb,Alive,Fair,Quercus palustris,...,27,11,QN49,Whitestone,4097300,New York,40.79411067,-73.81867946,1034455.70109,228644.837379
2,09/05/2015,204026,218365,"{'type': 'Point', 'coordinates': [-73.93660770...",3,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,50,18,BK90,East Williamsburg,3044900,New York,40.71758074,-73.9366077,1001822.83131,200716.891267
3,09/05/2015,204337,217969,"{'type': 'Point', 'coordinates': [-73.93445615...",10,0,OnCurb,Alive,Good,Gleditsia triacanthos var. inermis,...,53,18,BK90,East Williamsburg,3044900,New York,40.71353749,-73.93445616,1002420.35833,199244.253136
4,08/30/2015,189565,223043,"{'type': 'Point', 'coordinates': [-73.97597938...",21,0,OnCurb,Alive,Good,Tilia americana,...,44,21,BK37,Park Slope-Gowanus,3016500,New York,40.66677776,-73.97597938,990913.775046,182202.425999


To save time, I will save the datasets downloaded locally.

In [15]:
complaints_df.to_csv("data/complaints.csv")
tree_df.to_csv("data/tree.csv")


In [16]:
complaints_df = pd.read_csv("data/complaints.csv", index_col=0)
tree_df = pd.read_csv("data/tree.csv", index_col=0)

/var/folders/98/l9_9vlzx4vs8g303whfvnnjc0000gn/T/ipykernel_17023/3438034653.py:1: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  complaints_df = pd.read_csv("data/complaints.csv", index_col=0)


### Monthly average rents from Zillow

In [17]:
zillow_df = pd.read_csv("data/zillow_rent_data.csv")

In [18]:
zillow_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,City,Metro,CountyName,2015-01-31,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,1,77494,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Fort Bend County,1606.206406,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,3,77449,zip,TX,TX,Katy,"Houston-The Woodlands-Sugar Land, TX",Harris County,1257.814660,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,91733,5,77084,zip,TX,TX,Houston,"Houston-The Woodlands-Sugar Land, TX",Harris County,NaN,...,1701.217520,1706.900064,1706.067787,1723.722320,1735.484670,1752.132904,1756.990323,1754.429516,1757.602011,1755.031490
3,93144,6,79936,zip,TX,TX,El Paso,"El Paso, TX",El Paso County,NaN,...,1419.480272,1458.063897,1471.726681,1466.734658,1456.175660,1462.478506,1466.267391,1490.237063,1488.180414,1494.366097
4,62093,7,11385,zip,NY,NY,New York,"New York-Newark-Jersey City, NY-NJ-PA",Queens County,NaN,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783


In [19]:
# Choose necessary columns
necessary_columns = ['RegionID', 'State', 'City', 'RegionName', 'CountyName', '2018-10-31', '2018-11-30', '2018-12-31', '2019-01-31',
                    '2019-02-28', '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30', '2019-07-31','2019-08-31',
                    '2019-09-30','2019-10-31','2019-11-30','2019-12-31', '2020-01-31','2020-02-29','2020-03-31',
                    '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31',
                    '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31', '2021-04-30', '2021-05-31',
                    '2021-06-30', '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30', '2021-12-31',
                    '2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
                    '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
                    '2023-03-31', '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31', '2023-08-31', '2023-09-30']
zillow_df = zillow_df[necessary_columns]
zillow_df.head()

,RegionID,State,City,RegionName,CountyName,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,TX,Katy,77494,Fort Bend County,1590.488945,1594.393947,1598.275260,1604.520456,1610.434841,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,TX,Katy,77449,Harris County,1356.220524,1356.988632,1353.259235,1350.062265,1348.472443,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,91733,TX,Houston,77084,Harris County,1333.479680,1341.995595,1341.381470,1345.813983,1332.540790,...,1701.217520,1706.900064,1706.067787,1723.722320,1735.484670,1752.132904,1756.990323,1754.429516,1757.602011,1755.031490
3,93144,TX,El Paso,79936,El Paso County,NaN,NaN,NaN,NaN,NaN,...,1419.480272,1458.063897,1471.726681,1466.734658,1456.175660,1462.478506,1466.267391,1490.237063,1488.180414,1494.366097
4,62093,NY,New York,11385,Queens County,2419.898399,2420.804474,2420.254366,2400.882444,2419.392453,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783


In [20]:
# Normalize column names & types
zillow_df.columns = zillow_df.columns.str.lower()

zillow_df.head()

,regionid,state,city,regionname,countyname,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,...,2022-12-31,2023-01-31,2023-02-28,2023-03-31,2023-04-30,2023-05-31,2023-06-30,2023-07-31,2023-08-31,2023-09-30
0,91982,TX,Katy,77494,Fort Bend County,1590.488945,1594.393947,1598.275260,1604.520456,1610.434841,...,1994.653463,2027.438438,2042.237444,2049.325559,2016.531345,2023.438976,2031.558202,2046.144009,2053.486247,2055.771355
1,91940,TX,Katy,77449,Harris County,1356.220524,1356.988632,1353.259235,1350.062265,1348.472443,...,1749.697900,1738.217986,1747.305840,1758.407295,1758.891075,1762.980879,1771.751591,1779.338402,1795.384582,1799.631140
2,91733,TX,Houston,77084,Harris County,1333.479680,1341.995595,1341.381470,1345.813983,1332.540790,...,1701.217520,1706.900064,1706.067787,1723.722320,1735.484670,1752.132904,1756.990323,1754.429516,1757.602011,1755.031490
3,93144,TX,El Paso,79936,El Paso County,NaN,NaN,NaN,NaN,NaN,...,1419.480272,1458.063897,1471.726681,1466.734658,1456.175660,1462.478506,1466.267391,1490.237063,1488.180414,1494.366097
4,62093,NY,New York,11385,Queens County,2419.898399,2420.804474,2420.254366,2400.882444,2419.392453,...,2935.808220,2895.699421,2873.209025,2881.906361,2913.546218,2963.964134,3005.735342,3034.413822,3064.476503,3079.585783


In [21]:
# Normalize SRID
zillow_df.crs = "EPSG:4326"

In [22]:
zillow_df.to_csv('data/cleaned_zillow_rent_data.csv', index=False)

### Zip Code

In [23]:
pip install dbfread

Note: you may need to restart the kernel to use updated packages.


In [24]:
from dbfread import DBF

# Specify the path to your DBF file
dbf_path = 'data/nyc_zipcodes.dbf'

# Use dbfread to read the DBF file
zip_table = DBF(dbf_path)

# Convert the table to a pandas DataFrame
zip_code_df = pd.DataFrame(iter(zip_table))

# Display the DataFrame
zip_code_df.head()

,ZIPCODE,BLDGZIP,PO_NAME,POPULATION,AREA,STATE,COUNTY,ST_FIPS,CTY_FIPS,URL,SHAPE_AREA,SHAPE_LEN
0,11436,0,Jamaica,18681.0,2.269930e+07,NY,Queens,36,081,http://www.usps.com/,0.0,0.0
1,11213,0,Brooklyn,62426.0,2.963100e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0
2,11212,0,Brooklyn,83866.0,4.197210e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0
3,11225,0,Brooklyn,56527.0,2.369863e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0
4,11218,0,Brooklyn,72280.0,3.686880e+07,NY,Kings,36,047,http://www.usps.com/,0.0,0.0


In [25]:
# Remove unnecessary columns
keep_columns = ['ZIPCODE', 'PO_NAME', 'POPULATION', 'STATE', 'COUNTY']
zip_code_df = zip_code_df[keep_columns]
zip_code_df.head()

,ZIPCODE,PO_NAME,POPULATION,STATE,COUNTY
0,11436,Jamaica,18681.0,NY,Queens
1,11213,Brooklyn,62426.0,NY,Kings
2,11212,Brooklyn,83866.0,NY,Kings
3,11225,Brooklyn,56527.0,NY,Kings
4,11218,Brooklyn,72280.0,NY,Kings


In [26]:
# Normalize column names & types
zip_code_df.columns = zip_code_df.columns.str.lower()


In [28]:
gdf = gpd.read_file("data/nyc_zipcodes.shp")

# Check the current CRS (it might be None if not set)
print("Current CRS:", gdf.crs)

# Set the CRS to a known value (replace 'EPSG:4326' with your current CRS if known)
gdf.crs = "EPSG:4326"  # Replace with the actual CRS if known

# Check the updated CRS
print("Updated CRS:", gdf.crs)

# Now, you can transform the geometries to the desired CRS
gdf = gdf.to_crs("EPSG:4326")

# Check the final CRS
print("Final CRS:", gdf.crs)

Current CRS: None
Updated CRS: EPSG:4326
Final CRS: EPSG:4326




## Data Cleaning & Data Filtering

In [ ]:
tree_df.head()

# Storing Data

In [ ]:
!createdb FINAL_PROJECT
!psql --dbname FINAL_PROJECT -c 'CREATE EXTENSION postgis;'

In [ ]:
pip install psycopg2-binary

In [ ]:
import psycopg2

In [ ]:
conn = psycopg2.connect("dbname=FINAL_PROJECT user=postgres password=123456")
cur = conn.cursor()

In [ ]:
with open('schema.sql', 'w') as f:
    command = """
        CREATE TABLE zip_codes (
            zip_code Integer PRIMARY KEY,
            neighborhood TEXT,
            borough TEXT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE complaints (
            complaint_id Integer PRIMARY KEY,
            created_date TIMESTAMP,
            complaint_type TEXT,
            descriptor TEXT,
            zip_code Integer REFERENCES zip_codes(zip_code),
            latitude FLOAT,
            longitude FLOAT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE trees (
            tree_id Integer PRIMARY KEY,
            species TEXT,
            diameter_inches FLOAT,
            health TEXT,
            zip_code Integer REFERENCES zip_codes(zip_code),
            latitude FLOAT,
            longitude FLOAT,
            geometry GEOMETRY(Point, 4326)
        );

        CREATE TABLE average_rents (
            zip_code Integer PRIMARY KEY,
            year INT,
            avg_rent DECIMAL
        );
    """
    f.write(command)

In [ ]:
cur.execute(command)
conn.commit()

# Understanding Data

# Visualizing Data